**Seoul Bike Sharing Dataset 데이터셋**

서울시의 시간대별 자전거 대여 데이터를 포함하고 있으며, 날씨와 계절 요인이 자전거 대여량에 어떤 영향을 미치는지를 분석할 수 있는 시계열형 예측 데이터셋입니다.

제공된 학습용 데이터(bike_train.csv)를 이용하여 대여 자전거 대수(Rented_Bike_Count)를 예측하는 모델을 개발하고, 개발한 모델에 기반하여 평가용 데이터(bike_test.csv)에 적용하여 얻은 대여 자전거 대수 예측 값을 아래 [제출형식]에 따라 csv 파일로 생성하여 제출하시오.
- 예측 결과는 RMSE(Root Mean Squared Error) 평가지표에 따라 평가함
- 성능이 우수한 예측 모델을 구축하기 위해서는 데이터 정제, Feature Engineering, 하이퍼 파라미터(hyper parameter) 최적화, 모델 비교 등이 필요할 수 있음. 다만, 과적합에 유의하여야 함


[[제출 형식]]
- 가. CSV 파일명: result.csv(파일명에 디렉토리/폴더 지정불가
- 나. 예측 칼럼명 : pred
- 다. 제출 칼럼 개수 : pred 칼럼 1개
- 라. 평가용 데이터 개수와 예측 결과 데이터 개수 일치 : 2,716개

[[제공 데이터]]
- 데이터 목록
- bike_train.csv : 학습용 데이터, 6,044개
- bike_test.csv : 평가용 데이터, 2,716개
- 평가용 데이터는 'Rented_Bike_Count' 칼럼 미제공

[[ CSV 파일 제출 방법 ]]
- CSV 파일명 : result.csv
- 대여된 자전거수 칼럼명 : pred
- 제출 CSV 파일 형식 예시
        pred
         225.004207
        1995.239831
         724.008460
        1237.054652
         167.213520

* 소수점 아래자릿수는 별도 지정하지 않음
* pred 칼럼 데이터 수는 2,716

In [129]:
# 라이브러리
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import root_mean_squared_error as RMSE
pd.set_option('display.max_columns', 1000)

# 성능평가 사용자함수
def get_scores(model, x_train, x_test, y_train, y_test):
    A = model.score(x_train, y_train)
    B = model.score(x_test, y_test)
    y_pred1 = model.predict(x_train)
    y_pred2 = model.predict(x_test)
    C = RMSE(y_train, y_pred1)
    D = RMSE(y_test, y_pred2)
    return f"r2: {A:.4f}, {B:.4f}, RMSE: {C:.4f}, {D:.4f}"

# 데이터 불러오기
path = "https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/"
train = pd.read_csv(path + "bike_train.csv")
test = pd.read_csv(path + "bike_test.csv")
# print(train.shape, test.shape) # (6044, 14) (2716, 13)
# print(train.head(3), test.head(3), sep="\n")

# 데이터 전처리
X = train.drop(columns=['Rented_Bike_Count'])
Y = train['Rented_Bike_Count']
X_all = pd.concat([X, test])
# print(X_all.head())
# print(X_all.info())
columns_obj = X_all.select_dtypes(include='object').columns
# print(columns_obj)
for col in columns_obj:
    X_all[col] = LabelEncoder().fit_transform(X_all[col])
# print(X_all.head(3))
temp = StandardScaler().fit_transform(X_all)
X_all = pd.DataFrame(temp, columns=X_all.columns)
# print(X_all)

# 데이터 분할(학습용, 평가용)
X = X_all.iloc[:len(X), :]
X_submission = X_all.iloc[len(X):, :]
# print(X.shape, X_submission.shape) # (6044, 13) (2716, 13)
temp = train_test_split(X, Y, test_size=0.3, random_state=42)
x_train, x_test, y_train, y_test =temp
# print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
# (4230, 13) (1814, 13) (4230,) (1814,)

# 모델적합
model1 = LinearRegression().fit(x_train, y_train)
# print(get_scores(model1, x_train, x_test, y_train, y_test)) # r2: 0.5555, 0.5296, RMSE: 429.1367, 428.0350

# model2 = DecisionTreeRegressor(random_state=1234).fit(x_train, y_train)
# print(get_scores(model2, x_train, x_test, y_train, y_test)) # r2: 1.0000, 0.7564, RMSE: 0.0000, 309.4370
# print(model2.get_depth()) # 28
# for d in range(3, 14):
#     model2 = DecisionTreeRegressor(max_depth=d, random_state=1234).fit(x_train, y_train)
#     print(d, get_scores(model2, x_train, x_test, y_train, y_test))
#     # 3 r2: 0.5862, 0.5757, RMSE: 412.2902, 408.4000
model2 = DecisionTreeRegressor(max_depth=3, random_state=42).fit(x_train, y_train)
# print(get_scores(model2, x_train, x_test, y_train, y_test))
# r2: 0.5833, 0.5859, RMSE: 411.6091, 411.8306

# model3 = RandomForestRegressor(random_state=1234).fit(x_train, y_train)
# print(get_scores(model3, x_train, x_test, y_train, y_test))
# r2: 0.9796, 0.8715, RMSE: 91.4885, 224.7517
# for d in range(3, 16):
#     model3 = RandomForestRegressor(max_depth=d, random_state=42).fit(x_train, y_train)
#     print(d, get_scores(model3, x_train, x_test, y_train, y_test))
# 3 r2: 0.6304, 0.6342, RMSE: 387.6634, 387.0682
model3 = RandomForestRegressor(max_depth=3, random_state=42).fit(x_train, y_train)
# print(get_scores(model3, x_train, x_test, y_train, y_test))
# r2: 0.6304, 0.6342, RMSE: 387.6634, 387.06825

model4 = AdaBoostRegressor(random_state=42).fit(x_train, y_train)
# print(get_scores(model4, x_train, x_test, y_train, y_test))
# r2: 0.5804, 0.5695, RMSE: 413.0434, 419.9243

model5 = GradientBoostingRegressor(learning_rate=0.11, random_state=123).fit(x_train, y_train)
print(get_scores(model5, x_train, x_test, y_train, y_test))
# r2: 0.8596, 0.8310, RMSE: 241.2108, 256.5837

# 모델적용, 예측값, 제출파일 생성
# f_model = model3
# y_pred = f_model.predict(X_submission)
# pd.DataFrame({'pred': y_pred}).to_csv("result_03(1기).csv", index=False)

# # 제출파일 확인
# temp = pd.read_csv("result_03(1기).csv")
# print(temp['pred'].value_counts(normalize=True))
# print("=" * 20)
# print(Y.value_counts(normalize=True))

r2: 0.8640, 0.8371, RMSE: 235.1847, 258.2798
